# ER-Benchを動かしてみよう


## EQ-Benchのリポジトリのクローンとセットアップ

In [1]:
!git clone https://github.com/EQ-bench/EQ-Bench.git

Cloning into 'EQ-Bench'...
remote: Enumerating objects: 1123, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 1123 (delta 180), reused 166 (delta 142), pack-reused 902 (from 1)
Receiving objects: 100% (1123/1123), 16.82 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (688/688), done.


In [2]:
# EQ-bench requirements:
!pip install -q tqdm sentencepiece hf_transfer openai scipy torch peft bitsandbytes trl accelerate tensorboardX huggingface_hub anthropic scikit-learn matplotlib google-generativeai
# These are for qwen models:
# !pip install -q einops transformers_stream_generator==0.0.4 deepspeed tiktoken git+https://github.com/Dao-AILab/flash-attention.git auto-gptq optimum
# These are for uploading results
# !pip install -q gspread oauth2client firebase_admin
# Install latest transformers from source last
!pip install -q git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 require

In [3]:
!pip install anthropic

In [4]:
%cd EQ-Bench

/content/EQ-Bench


In [5]:
!ls

config.cfg		 eq-bench.py		lib		       results
data			 images			LICENSE
eq_bench_prompts_v1.txt  install_reqs.sh	ooba_quick_install.sh
eq_bench_prompts_v2.txt  instruction-templates	README.md


## chat templateの準備

EQ-Benchはリポジトリ内に独自のchat templateを保存しているため、非対応のテンプレートを用いたモデルの場合はchat templateを作成して保存しておく必要があります。

今回使用するllama3は対応していないようなので、chat templateを呼び出して加工・保存します。

In [22]:
model_path = "tokyotech-llm/Llama-3.1-Swallow-8B-v0.2" #@param {type:"string"}

In [23]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [24]:
def masked_replace(s_input, s_replace_target, s_replace_to, s_mask_target):
  s_mask = "maskBSSBDfsdbhfabkfb"
  return s_input.replace(s_mask_target, s_mask).replace(s_replace_target, s_replace_to).replace(s_mask, s_mask_target)

ロールは以下のような置き換えが必要です。

- **name_template_user**: `user` -> `<|user|>`
- **name_template_assistant**: `assistant` -> `<|bot|>`

promptは以下のような置き換えが必要です。

- **prompt_template_system**: `system prompt` -> `<|system-message|>`
- **prompt_template_user**: `user prompt` -> `<|user-message|>`
- **prompt_template_assist**: `assistant response` -> `<|bot-message|>`


In [25]:
name_template_system = "system" #@param {type:"string"}
name_template_user = "user" #@param {type:"string"}
name_template_assist = "assistant" #@param {type:"string"}

prompt_template_system = "<|system-message|>" #@param {type:"string"}
prompt_template_user = "<|user-message|>" #@param {type:"string"}
prompt_template_assist = "<|bot-message|>" #@param {type:"string"}

chat_dummy = [
    {"role": name_template_system, "content": prompt_template_system},
    {"role": name_template_user, "content": prompt_template_user},
    {"role": name_template_assist, "content": prompt_template_assist},
]
chat_template = tokenizer.apply_chat_template(chat_dummy, tokenize=False)
#print(chat_template)

system_dummy = [
    {"role": "system", "content": "<|system-message|>"},
]
system_template = tokenizer.apply_chat_template(system_dummy, tokenize=False)

chat_template_save = chat_template.replace("\n","\\n").replace(system_template.replace("\n","\\n"), "")
chat_template_save = masked_replace(chat_template_save, name_template_user, "<|user|>", "<|user-message|>")
chat_template_save = masked_replace(chat_template_save, name_template_assist, "<|bot|>", "<|bot-message|>")
# print(chat_template)

# system_template = masked_replace(system_template, name_template_system, "<|bot|>", "<|system-message|>")
system_template_save = system_template.replace("\n","\\n")
# print("=====")
# print(system_template)


name_template_save = "llama3" #@param {type:"string"}



上記の通り置換したうえで、ここで表示されているテンプレートが以下の通りとなっている必要があります。

```
user: {name_template_user} # テンプレート上のuser名
bot: {name_template_assistant} # テンプレート上のLLM側の名前
turn_template: # userとassistantだけの文のchat template
context: # system templateのみ
```

tokenizer内のchat templateも参考にしてください。

In [26]:
print(chat_template)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|system-message|><|eot_id|><|start_header_id|>user<|end_header_id|>

<|user-message|><|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|bot-message|><|eot_id|>


In [27]:
template_save = f"user: \"{name_template_user}\"\nbot: \"{name_template_assist}\"\nturn_template: \"{chat_template_save}\"\ncontext: \"{system_template_save}\"\n"
# print("=====")
print(template_save)


user: "user"
bot: "assistant"
turn_template: "<|start_header_id|><|user|><|end_header_id|>\n\n<|user-message|><|eot_id|><|start_header_id|><|bot|><|end_header_id|>\n\n<|bot-message|><|eot_id|>"
context: "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|system-message|><|eot_id|>"



In [28]:
with open(f"/content/EQ-Bench/instruction-templates/{name_template_save}.yaml", "w", encoding="utf-8") as f:
  f.write(template_save)

## 設定ファイル(config.cfg)の編集

次に、EQ-Benchを行うためのモデル設定や評価用のAPI設定などを行います。

In [15]:
import configparser

In [16]:
config_ini = configparser.ConfigParser(allow_no_value=True)
config_ini.read('config.cfg', encoding='utf-8')

['config.cfg']

In [1]:
#@markdown ## [OpenAI]

#@markdown Optional. Set these if you are testing OpenAI models.
api_key =  "" #@param {type:"string"}
config_ini["OpenAI"]["api_key"] = api_key

#@markdown Optional. Set this if you are using an alternative OpenAI compatible endpoint, like ollama running locally

#@markdown openai_compatible_url = http://localhost:8080/v1/

#@markdown openai_compatible_url = https://api.together.xyz

#@markdown ## [Huggingface]

#@markdown Optional. This allows the script to download gated models.
access_token =  "" #@param {type:"string"}
config_ini["Huggingface"]["access_token"] = access_token

#@markdown Optional. This is where models will be downloaded to.

#@markdown - defaults to ~/.cache/huggingface/hub
cache_dir =   "" #@param {type:"string"}
config_ini["Huggingface"]["cache_dir"] = cache_dir

#@markdown ## [Results upload]

#@markdown Optional. Set this to allow uploading of results to a google sheets spreadsheet.

#@markdown Note: this feature requires extra configuration (see README).
google_spreadsheet_url =  "" #@param {type:"string"}
config_ini["Results upload"]["google_spreadsheet_url"] = google_spreadsheet_url

#@markdown ## [Creative Writing Benchmark]
#@markdown judge_model_api: openai / mistralai / anthropic
judge_model_api =  "anthropic" #@param ["openai", "mistralai", "anthropic"] {allow-input: true}
#@markdown `claude-3-5-sonnet-20241022`が推奨されています。
judge_model =  "claude-3-5-sonnet-20241022" #@param {type:"string"}
judge_model_api_key =  "api_key" #@param {type:"string"}


config_ini["Creative Writing Benchmark"]["judge_model_api"] = judge_model_api
config_ini["Creative Writing Benchmark"]["judge_model"] = judge_model
config_ini["Creative Writing Benchmark"]["judge_model_api_key"] = judge_model_api_key

#@markdown ## [Options]

#@markdown Set to true or false
trust_remote_code = "true" #@param ["true", "false"] {allow-input: true}
config_ini["Options"]["trust_remote_code"] = trust_remote_code

#@markdown ## [Oobabooga config]

#@markdown e.g. ~/text-generation-webui/start_linux.sh
ooba_launch_script = "" #@param {type:"string"}
config_ini["Oobabooga config"]["ooba_launch_script"] = ooba_launch_script

#@markdown Specify any additional oobabooga launch params (this can be overridden on a per-model basis).

#@markdown e.g.:

#@markdown --auto-devices --loader llama.cpp
ooba_params_global = "" #@param {type:"string"}
config_ini["Oobabooga config"]["ooba_params_global"] = ooba_params_global

#@markdown Set to true or false. Setting to "true" only supports linux (and possibly mac).

#@markdown If set to false, you must launch ooba yourself with --api flag and load the model yourself before running the benchmark.

#@markdown If you are launching ooba yourself, the model_path param will be ignored, and all scheduled ooba benchmark runs will use the same model.
automatically_launch_ooba = "false"  #@param ["true", "false"] {allow-input: true}
config_ini["Oobabooga config"]["automatically_launch_ooba"] = automatically_launch_ooba

#@markdown Ooba api request timeout in seconds (default 120). Set higher if you are expecting long inference times.
ooba_request_timeout = "600" #@param {type:"string"}
config_ini["Oobabooga config"]["ooba_request_timeout"] = ooba_request_timeout

NameError: name 'config_ini' is not defined

In [18]:
# 書き込み
with open("config.cfg", "w", encoding="utf-8") as f: # config.cfgを上書き
    config_ini.write(f)

ベンチマーク用の設定をします。

In [29]:
#@markdown ## [Benchmarks to run]

#@markdown **run_id**
#@markdown   - ユーザーが任意で指定する、ベンチマーク実行を識別するための名前。
#@markdown   - 出力ファイルやログ内で、このIDを用いて結果を区別します。
run_id = "myrun1" #@param {type:"string"}
# if run_id:
#   config_ini["Benchmarks to run"]["run_id"] = run_id

#@markdown **instruction_template**
#@markdown   - プロンプト形式を定義するテンプレート名（`.yaml`拡張子なし）
#@markdown   - `instruction-templates`ディレクトリ内にある対応する`.yaml`ファイルに準拠したフォーマットでLLMへの指示（プロンプト）を生成します。
instruction_template = "" #@param {type:"string"}
instruction_template = instruction_template if instruction_template != "" else name_template_save

# if instruction_template:
#   config_ini["Benchmarks to run"]["instruction_template"] = instruction_template

#@markdown **model_path**
#@markdown   - 使用するモデルを指定するパスまたはモデルID。
#@markdown   - Hugging FaceのモデルID（例：`meta-llama/Llama-2-7b-chat-hf`）
#@markdown   - ローカルファイルパス（例：`~/my_local_model`）
#@markdown   - OpenAIや他のAPI互換モデルの場合は対応するモデル名（例：`gpt-4-0613`）
#@markdown   - inference_engineによって解釈が異なります。
#@markdown   - **model_path**は最初のmodel_pathを設定します。
# model_path = "unsloth/Meta-Llama-3.1-8B-Instruct" #@param {type:"string"}
# if model_path:
#   config_ini["Benchmarks to run"]["model_path"] = model_path

#@markdown **lora_path**（オプション）
#@markdown   - LoRAアダプタを使用する場合、そのアダプタファイルまたはディレクトリへのパス。
#@markdown   - 指定しない場合は空欄で可。
lora_path = "" #@param {type:"string"}
# if lora_path:
#   config_ini["Benchmarks to run"]["lora_path"] = lora_path

#@markdown **quantization**
#@markdown   - モデルロード時の量子化設定。
#@markdown   - `8bit`, `4bit`, `None` (または空)が有効。
#@markdown   - BitsAndBytesによるGPUメモリ削減のために使用します。
quantization = "8bit"  #@param ["None", "4bit", "8bit"] {allow-input: true}
# if quantization:
#   config_ini["Benchmarks to run"]["quantization"] = quantization

#@markdown **n_iterations**
#@markdown   - ベンチマークを何回繰り返すかを指定する整数値。
#@markdown   - 出力スコアは、この回数分の実行結果を平均したものが用いられます。
#@markdown   - 例：`3` とすれば3回実行して平均スコアを求めます。
n_iterations = "3" #@param {type:"string"}
# if n_iterations:
#   config_ini["Benchmarks to run"]["n_iterations"] = n_iterations

#@markdown **inference_engine**
#@markdown   - 推論エンジンを指定する文字列。
#@markdown   - 例：`transformers`, `ooba`, `openai`, `llama.cpp`, `anthropic`, `mistralai`, `gemini`など
#@markdown   - モデルの実行方法（ローカル/外部APIなど）を決定します。
inference_engine = "transformers" #@param {type:"string"}
# if inference_engine:
#   config_ini["Benchmarks to run"]["inference_engine"] = inference_engine

#@markdown **ooba_params**（オプション）
#@markdown   - inference_engineが`ooba`の場合のみ有効で、モデルロード時や推論時の追加パラメータを指定します。
#@markdown   - `--loader transformers --n_ctx 1024 --n-gpu-layers -1` といった形式で、oobaboogaのWebUI起動パラメータをカンマ区切りで記述します。
ooba_params = "" #@param {type:"string"}
# if ooba_params:
#   config_ini["Benchmarks to run"]["ooba_params"] = ooba_params

#@markdown **downloader_filters**（オプション）
#@markdown   - Hugging Faceからモデルをダウンロードする際のフィルタ設定。
#@markdown   - `--include`や`--exclude`パターンを使用し、特定のファイルのみダウンロードしたり除外したりできます。
#@markdown   - 例：`--include ["*Q3_K_M.gguf", "*.json"]`
#@markdown   - 記載しない場合は空欄で可。
downloader_args = "" #@param {type:"string"}
# if downloader_args:
#   config_ini["Benchmarks to run"]["downloader_args"] = downloader_args




setting_bench = ",".join([run_id,instruction_template,model_path,lora_path,quantization,n_iterations,inference_engine,ooba_params,downloader_args])
print(setting_bench)


myrun1,llama3,unsloth/Meta-Llama-3.1-8B-Instruct,,8bit,3,transformers,,


`config.cfg`にベンチマーク用の設定`setting_bench`を書き込む

In [30]:
with open("config.cfg", "r", encoding="utf-8") as reader:
  config_txt = reader.read()

section_benchmark_setting = "[Benchmarks to run]"
config_txt = config_txt.replace(section_benchmark_setting, section_benchmark_setting+"\n"+setting_bench)

with open("config.cfg", "w", encoding="utf-8") as writer:
  writer.write(config_txt)

## ベンチマーク実行部分

In [ ]:
!python eq-bench.py --benchmarks creative-writing

2024-12-18 15:29:47.099316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 15:29:47.118718: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 15:29:47.124487: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 15:29:47.138388: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-18 15:29:48.433841: W tensorflow/comp

In [21]:
!python eq-bench.py --benchmarks eq-bench